In [ ]:
import threading
import cv2
import numpy as np
import tensorflow as tf
from gtts import gTTS
import pygame
import os
import speech_recognition as sr

# === Load model dan class mapping ===
model_path = r'C:\Users\edwin\OneDrive - Bina Nusantara\PPTI - CAWU 3\Artificial Intelligence\project\KLASIFIKASI_UANG_KERTAS_RUPIAH\MODEL3 - DenseNet\final_model_densenet201_2.h5'
class_mapping_path = r'C:\Users\edwin\OneDrive - Bina Nusantara\PPTI - CAWU 3\Artificial Intelligence\project\KLASIFIKASI_UANG_KERTAS_RUPIAH\MODEL3 - DenseNet\class_mapping-2.npy'

perintah_suara = ""

daftar_perintah = ['tes', 'test', 'ambil gambar', 'ini berapa', 'berapa ini', 'berapa', 'rupai', 'rup ai', 'apa ini', 'klasifikasi']

model = tf.keras.models.load_model(model_path)
class_mapping = np.load(class_mapping_path, allow_pickle=True).item()
class_names = list(class_mapping.values())

def bicara(teks):
    tts = gTTS(text=teks, lang='id')
    filename = "temp_suara.mp3"
    tts.save(filename)

    pygame.mixer.init()
    pygame.mixer.music.load(filename)
    pygame.mixer.music.play()

    while pygame.mixer.music.get_busy():
        continue

    pygame.mixer.quit()
    os.remove(filename)

def prediksi_gambar(img):
    if img.shape[2] == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    img = cv2.resize(img, (224, 224))
    img = tf.keras.applications.mobilenet_v2.preprocess_input(img)
    input_tensor = np.expand_dims(img, axis=0)

    prediksi = model.predict(input_tensor, verbose=0)[0]
    idx_kelas = np.argmax(prediksi)
    kepercayaan = prediksi[idx_kelas]
    return class_names[idx_kelas], kepercayaan

def dengarkan_perintah():
    pengenal = sr.Recognizer()
    mic = sr.Microphone()

    with mic as sumber:
        print("🎙️ Mendengarkan... (ucapkan: " + ", ".join(f"'{p}'" for p in daftar_perintah) + ")")
        pengenal.adjust_for_ambient_noise(sumber)
        try:
            audio = pengenal.listen(sumber, timeout=5)
            perintah = pengenal.recognize_google(audio, language="id-ID").lower()
            print("Kamu mengatakan:", perintah)
            return perintah
        except sr.WaitTimeoutError:
            print("Tidak ada suara terdeteksi.")
        except sr.UnknownValueError:
            print("Maaf, saya tidak mengerti.")
        except sr.RequestError as e:
            print("Error pada layanan pengenalan suara:", e)
    return ""

def thread_dengarkan():
    global perintah_suara
    while True:
        if perintah_suara == "": 
            perintah = dengarkan_perintah()
            for p in daftar_perintah:
                if p in perintah:
                    perintah_suara = perintah
                    break

thread = threading.Thread(target=thread_dengarkan, daemon=True)
thread.start()

cap = cv2.VideoCapture(0)

print("Program dimulai. Ucapkan 'ambil gambar' kapan saja, atau tekan 'q' untuk keluar.")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Gagal menangkap gambar.")
        break

    cv2.putText(frame, "Ucapkan sesuatu | Tekan 'q' untuk keluar", (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
    cv2.imshow("Klasifikasi Uang Kertas", frame)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        print("Keluar dari program.")
        break

    if perintah_suara:
        print("Perintah terdeteksi:", perintah_suara)
        print("Mengambil gambar...")
        nama_kelas, kepercayaan = prediksi_gambar(frame)
        teks_label = f"{nama_kelas} ({kepercayaan:.2%})"
        print("Prediksi:", teks_label)

        hasil_frame = frame.copy()
        cv2.putText(hasil_frame, teks_label, (30, 80),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.imshow("Klasifikasi Uang Kertas", hasil_frame)

        bicara(nama_kelas)
        cv2.waitKey(2000)

        perintah_suara = ""

cap.release()
cv2.destroyAllWindows()

c:\Users\edwin\anaconda3\envs\ai-klasifikasi-mata-uang-rupiah\lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


pygame 2.6.1 (SDL 2.28.4, Python 3.10.18)
Hello from the pygame community. https://www.pygame.org/contribute.html


🎙️ Mendengarkan... (ucapkan: 'tes', 'test', 'ambil gambar', 'ini berapa', 'berapa ini', 'berapa', 'rupai', 'rup ai', 'apa ini', 'klasifikasi')
Program dimulai. Ucapkan 'ambil gambar' kapan saja, atau tekan 'q' untuk keluar.
Maaf, saya tidak mengerti.
🎙️ Mendengarkan... (ucapkan: 'tes', 'test', 'ambil gambar', 'ini berapa', 'berapa ini', 'berapa', 'rupai', 'rup ai', 'apa ini', 'klasifikasi')
Maaf, saya tidak mengerti.
🎙️ Mendengarkan... (ucapkan: 'tes', 'test', 'ambil gambar', 'ini berapa', 'berapa ini', 'berapa', 'rupai', 'rup ai', 'apa ini', 'klasifikasi')
Maaf, saya tidak mengerti.
🎙️ Mendengarkan... (ucapkan: 'tes', 'test', 'ambil gambar', 'ini berapa', 'berapa ini', 'berapa', 'rupai', 'rup ai', 'apa ini', 'klasifikasi')
Kamu mengatakan: apa ini
Perintah terdeteksi: apa ini
Mengambil gambar...
Prediksi: seribu (99.76%)
🎙️ Mendengarkan... (ucapkan: 'tes', 'test', 'ambil

Kamu mengatakan: makasih ya aku ganti coba ganti itu yang
🎙️ Mendengarkan... (ucapkan: 'tes', 'test', 'ambil gambar', 'ini berapa', 'berapa ini', 'berapa', 'rupai', 'rup ai', 'apa ini', 'klasifikasi')
Maaf, saya tidak mengerti.
🎙️ Mendengarkan... (ucapkan: 'tes', 'test', 'ambil gambar', 'ini berapa', 'berapa ini', 'berapa', 'rupai', 'rup ai', 'apa ini', 'klasifikasi')
Kamu mengatakan: kanjeng mas
🎙️ Mendengarkan... (ucapkan: 'tes', 'test', 'ambil gambar', 'ini berapa', 'berapa ini', 'berapa', 'rupai', 'rup ai', 'apa ini', 'klasifikasi')
Maaf, saya tidak mengerti.
🎙️ Mendengarkan... (ucapkan: 'tes', 'test', 'ambil gambar', 'ini berapa', 'berapa ini', 'berapa', 'rupai', 'rup ai', 'apa ini', 'klasifikasi')
Maaf, saya tidak mengerti.
🎙️ Mendengarkan... (ucapkan: 'tes', 'test', 'ambil gambar', 'ini berapa', 'berapa ini', 'berapa', 'rupai', 'rup ai', 'apa ini', 'klasifikasi')
Maaf, saya tidak mengerti.
🎙️ Mendengarkan... (ucapkan: 'tes', 'test', 'ambil gambar', 'ini berapa', 'berapa ini', 'b